# Traveling Salesman Problem (TSP)

**Objectives**

- Introduce students to a real world problem solved by OR practitioners
- Demonstrate the use of heuristics to obtain good solutions to optimization problems
- Give students an appreciation of the difficulty of solving optimization problems exactly

**Reading:** Read Handout 2 on the traveling salesman problem.

**Brief description:** Finding an optimal solution to a Traveling Salesman Problem, and proving that it is, in fact, an optimal solution, is a difficult task. In practice, when a feasible solution to a difficult problem needs to be provided quickly, one often resorts to using heuristics, i.e., procedures for generating feasible solutions, or improving existing ones, that can be executed quickly and, hopefully, produce a pretty good result. In this lab we will consider several such heuristic procedures for the TSP.

*TSP VLSI instances adapted from [TSPLIB](http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/) and [Bonn Institute](http://www.math.uwaterloo.ca/tsp/vlsi/index.html)*

<font color='blue'> <b>Solutions are shown blue.</b> </font> <br>
<font color='red'> <b>Instuctor comments are shown in red.</b> </font>

<font color='red'>A tool we might want to use: [TSP DIY](https://www.math.uwaterloo.ca/tsp/app/diy.html)  </font>

<font color='red'> Some explanation of what imports are here.</font>

In [1]:
# imports -- don't forget to run this cell
from tsp import *
from bokeh.io import output_notebook
from random import randrange
output_notebook()

Loading BokehJS ...

In [2]:
nodes, G = tsp_grid_instance(15,15,manhattan=False)
plot_tour(nodes, G, nearest_neighbor(G))
plot_tour(nodes, G, furthest_insertion(G))
plot_tour(nodes, G, solve_tsp(G))

In [3]:
nodes, G = tsp_grid_instance(5,5,manhattan=False)
plot_tsp_heuristic(nodes, G, heuristic='nearest_neighbor', initial=0)

In [4]:
n = 10
random_neighbor_total = 0
nearest_neighbor_total = 0
nearest_insertion_total = 0
furthest_insertion_total = 0
for i in range(n):
    nodes, G = tsp_grid_instance(6,8,True)
    random_neighbor_total += tour_cost(G, random_neighbor(G))
    nearest_neighbor_total += tour_cost(G, nearest_neighbor(G))
    nearest_insertion_total += tour_cost(G, nearest_insertion(G))
    furthest_insertion_total += tour_cost(G, furthest_insertion(G))
print("Heuristic Averages:")
print("Random Neighbor: %s" % (random_neighbor_total / n))
print("Nearest Neighbor: %s" % (nearest_neighbor_total / n))
print("Nearest Insertion: %s" % (nearest_insertion_total / n))
print("Furthest Insertion: %s" % (furthest_insertion_total / n))

Heuristic Averages:
Random Neighbor: 220.8
Nearest Neighbor: 62.2
Nearest Insertion: 61.8
Furthest Insertion: 57.0


In [5]:
nodes = pd.read_csv('data/xqg237.csv', index_col=0)
G = create_G(nodes, manhattan=False)
plot_tour(nodes, G, solve_tsp(G))

In [6]:
# TODO: Finish generating an etching instance
nodes = pd.read_csv('data/xqf131.csv', index_col=0)
nodes = nodes.rename(columns={'x':'x_start', 'y':'y_start'})
nodes['x_end'] = 0
nodes['y_end'] = 0
for index, row in nodes.iterrows():
    rand = randrange(4)
    length = randrange(2,4)
    if rand == 0:
        nodes.at[index,'x_end'] = row['x_start'] + length
        nodes.at[index,'y_end'] = row['y_start']
    if rand == 1:
        nodes.at[index,'x_end'] = row['x_start']
        nodes.at[index,'y_end'] = row['y_start'] + length
    if rand == 2:
        nodes.at[index,'x_end'] = row['x_start'] - length
        nodes.at[index,'y_end'] = row['y_start']
    if rand == 3:
        nodes.at[index,'x_end'] = row['x_start']
        nodes.at[index,'y_end'] = row['y_start'] - length

In [7]:
G = create_vlsi_G(nodes)
plot_vlsi_tour(nodes,G,solve_tsp(G))